<a href="https://colab.research.google.com/github/atejap05/pos_data_science_furg/blob/main/disciplinas/Machine_Learning_I/semana04/Furg_ECD_04_Machine_Learning_I_Regress%C3%A3o.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Curso de Especialização em Ciência de Dados - FURG
## Machine Learning I - Regressão
### Prof. Marcelo Malheiros

Código adaptado de Aurélien Geron (licença Apache-2.0)

---

# Inicialização

Aqui importamos as bibliotecas fundamentais de Python para este _notebook_:

- NumPy: suporte a vetores, matrizes e operações de Álgebra Linear
- Matplotlib: biblioteca de visualização de dados
- Scikit-Learn: biblioteca com algoritmos de Machine Learning

In [ ]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
import sklearn

# Regressão Linear

Primeiro vamos criar um conjunto sintético de dados, com um "ruído" aleatório adicionado à reta $y = 4 + 3 * X$.

In [ ]:
np.random.seed(42)

X = 2 * np.random.rand(100, 1)
y = 4 + 3 * X + np.random.randn(100, 1)

In [ ]:
plt.plot(X, y, 'b.')
plt.xlabel('$x_1$', fontsize=18)
plt.ylabel('$y$', rotation=0, fontsize=18)
plt.axis([0, 2, 0, 15])
plt.show()

## Treinamento usando a solução de forma fechada

Isso quer dizer que o modelo de regressão linear vai criar uma equação completa para fazer a previsão, usando todos os dados de treinamento de uma só vez.

Agora vamos treinar o modelo com as _features_ em `X` e os rótulos em `y`:

In [ ]:
from sklearn.linear_model import LinearRegression

lin_reg = LinearRegression()
lin_reg.fit(X, y)

O modelo treinado aprendeu os parâmetros abaixo, da reta original $y = 4 + 3 * X$. A discrepância se dá por conta do ruído adicionado.

In [ ]:
print('interseção: ', lin_reg.intercept_[0])
print('coeficiente:' , lin_reg.coef_[0, 0])

Agora vamos plotar a reta dada pelo modelo de regressão, usando o resultado das previsões para 0 e 2 (canto esquerdo e direito do gráfico, respectivamente).

In [ ]:
X_new = np.array([[0], [2]])
y_predict = lin_reg.predict(X_new)

plt.plot(X_new, y_predict, "r-")
plt.plot(X, y, "b.")
plt.axis([0, 2, 0, 15])
plt.show()

## Funcionamento do método de descida do gradiente

Aqui o processo é incremental, começando longe de uma boa aproximação, mas gradualmente chegando perto de uma boa solução à medida que novos dados de treinamento são processados.

Este é um exemplo de um método *iterativo* de otimização.

Não se preocupe com os detalhes do código abaixo, este é só para demonstrar a estrutura do método de descida do gradiente!

In [ ]:
# taxa de aprendizado (learning rate)
eta = 0.1  

# número de passos (iterações)
n_iterations = 1000

# ponto inicial aleatório
theta = np.random.randn(2,1)

X_b = np.c_[np.ones((100, 1)), X]
for iteration in range(n_iterations):
    gradients = 2/100 * X_b.T.dot(X_b.dot(theta) - y)
    theta = theta - eta * gradients

In [ ]:
print('interseção: ',  theta[0, 0])
print('coeficiente:' , theta[1, 0])

O código a seguir é para gerar as figuras abaixo, que ilustram o proceso de descida do gradiente. Não se preocupe com os detalhes.

In [ ]:
def plot_gradient_descent(theta, eta, theta_path=None):
    m = len(X_b)
    plt.plot(X, y, 'b.')
    n_iterations = 1000
    for iteration in range(n_iterations):
        if iteration < 10:
            y_predict = X_new_b.dot(theta)
            style = 'b-' if iteration > 0 else 'r--'
            plt.plot(X_new, y_predict, style)
        gradients = 2/m * X_b.T.dot(X_b.dot(theta) - y)
        theta = theta - eta * gradients
        if theta_path is not None:
            theta_path.append(theta)
    plt.xlabel('$x_1$', fontsize=18)
    plt.axis([0, 2, -2, 15])
    plt.title('$\eta = {}$'.format(eta), fontsize=16)

Na figura mais à esquerda a taxa de aprendizado é $\eta=0.02$, causando uma **convergência muito lenta**.

Na figura do meio, a taxa é $\eta=0.1$, dando a **convergência correta**.

Na figura mais à direita, a taxa é muito grande com $\eta=0.5$, o que causa **divergência**. Ou seja, o método numérico não converge para a solução desejada

A linha vermelha representa o ponto inicial.

In [ ]:
np.random.seed(42)
theta = np.random.randn(2,1) # ponto inicial aleatório

X_new_b = np.c_[np.ones((2, 1)), X_new]
plt.figure(figsize=(15,3))
plt.subplot(131); plot_gradient_descent(theta, eta=0.02)
plt.ylabel('$y$', rotation=0, fontsize=18)
plt.subplot(132); plot_gradient_descent(theta, eta=0.1)
plt.subplot(133); plot_gradient_descent(theta, eta=0.5)
plt.show()

## Treinamento usando o método de descida do gradiente

Aqui vamos usar o algoritmo `SGDRegressor`, que randomiza a escolha de instâncias a cada passo (sendo uma técnica de Descida de Gradiente Estocástico).

O parâmetro `eta0` é a taxa de aprendizado, enquando `max_iter` é o limite para o número de passos.

A vantagem é poder fugir com mais facilidade de mínimos locais e encontrar os parâmetros que minimizam globalmente o erro, encontrando a melhor solução para a regressão linear.

In [ ]:
from sklearn.linear_model import SGDRegressor

sgd_reg = SGDRegressor(eta0=0.1, penalty=None, random_state=42, max_iter=1000)
sgd_reg.fit(X, y.ravel())
print('interseção: ',  sgd_reg.intercept_[0])
print('coeficiente:' , sgd_reg.coef_[0])

# Regressão polinomial

Vamos criar um conjunto sintético de dados, com um "ruído" aleatório adicionado à curva $y = 2 + X + 0.5 X^2$.

In [ ]:
np.random.seed(42)

X = 6 * np.random.rand(100, 1) - 3
y = 2 + X + 0.5 * X**2 + np.random.randn(100, 1)

In [ ]:
plt.plot(X, y, 'b.')
plt.xlabel('$x_1$', fontsize=18)
plt.ylabel('$y$', rotation=0, fontsize=18)
plt.axis([-3, 3, 0, 10])
plt.show()

O trecho mostra o recurso de transformação de dados chamado `PolynomialFeatures`, que automatiza a inserção de novas _features_ no _dataset_ original.

No caso, vamos inserir apenas um termo quadrático $x^2$ para cada feature $x$.

In [ ]:
from sklearn.preprocessing import PolynomialFeatures

poly_features = PolynomialFeatures(degree=2, include_bias=False)
X_poly = poly_features.fit_transform(X)

In [ ]:
print('instância #0 original:  ', X[0])
print('instância #0 modificada:', X_poly[0])

Compare os parâmetros aprendidos com a equação da curva original, $y = 2 + X + 0.5 X^2$.

In [ ]:
# ajuste pelo algoritmo de regressão linear (usando solução de forma fechada)
lin_reg = LinearRegression()
lin_reg.fit(X_poly, y)
print('interseção:  ',  lin_reg.intercept_[0])
print('coeficientes:' , lin_reg.coef_[0])

In [ ]:
# exibição da curva ajustada
X_new = np.linspace(-3, 3, 100).reshape(100, 1)
X_new_poly = poly_features.transform(X_new)
y_new = lin_reg.predict(X_new_poly)
plt.plot(X, y, 'b.')
plt.plot(X_new, y_new, 'r-', linewidth=2, label='previsões')
plt.xlabel('$x_1$', fontsize=18)
plt.ylabel('$y$', rotation=0, fontsize=18)
plt.legend(loc='upper left', fontsize=14)
plt.axis([-3, 3, 0, 10])
plt.show()

Note que uma regressão polinomial é capaz de encontrar relacionamentos entre features (o que um modelo de regressão linear simples não pode fazer).

Isso é possível porque `PolynomialFeatures` também pode adicionar todas as combinações de _features_ até o grau determinado. Por exemplo, se houvesse duas _features_ $a$ e $b$, `PolynomialFeatures` com `degree=3` não apenas adicionaria as _features_ $a^2$, $a^3$, $b^2$ e $b^3$, mas também as combinações $ab$, $a^2b$ e $ab^2$.

# Regressão logística

In [ ]:
t = np.linspace(-10, 10, 100)
sig = 1 / (1 + np.exp(-t))
plt.figure(figsize=(9, 3))
plt.plot([-10, 10], [0, 0], "k-")
plt.plot([-10, 10], [0.5, 0.5], "k:")
plt.plot([-10, 10], [1, 1], "k:")
plt.plot([0, 0], [-1.1, 1.1], "k-")
plt.plot(t, sig, "b-", linewidth=2, label=r"$\sigma(t) = \frac{1}{1 + e^{-t}}$")
plt.xlabel("t")
plt.legend(loc="upper left", fontsize=20)
plt.axis([-10, 10, -0.1, 1.1])
plt.show()

## Obter os dados

Nesta atividade iremos usar o conjunto de dados IRIS, que é um conjunto famoso que contém o comprimento e largura das sépalas e das pétalas de 150 flores de íris de três espécies diferentes (_Iris setosa_, _Iris versicolor_ e _Iris virginica).

Este _dataset_ já faz parte da biblioteca Scikit-Learn, e pode ser importado diretamente.

In [ ]:
from sklearn import datasets

iris = datasets.load_iris()
list(iris.keys())

Agora será construido um classificador usando regressão logística para detectar o tipo _Iris virginica_ com base apenas na _feature_ de largura da pétala.

In [ ]:
X = iris['data'][:, 3:]                   # largura da pétala
y = (iris['target'] == 2).astype(np.int)  # 1 se é Iris virginica, caso contrário 0

In [ ]:
from sklearn.linear_model import LogisticRegression

log_reg = LogisticRegression(random_state=42)
log_reg.fit(X, y)

Um classificador logístico é baseado em probabilidades, então vamos examinar o modelo e verificar as probabilidades estimadas para flores com larguras de pétalas variando de 0 a 3 cm.

Note que `log_reg.predict_proba()` devolve probabilidades no intervalo [0, 1], enquanto `log_reg.predict()` retorna a ser da classe (classe positiva ou 1) ou não ser da classe (negativa ou 0).

In [ ]:
X_new = np.linspace(0, 3, 1000).reshape(-1, 1)
y_proba = log_reg.predict_proba(X_new)
limiar = X_new[y_proba[:, 1] >= 0.5][0]

plt.plot(X_new, y_proba[:, 1], 'g-',  linewidth=2, label='Iris virginica')
plt.plot(X_new, y_proba[:, 0], 'b--', linewidth=2, label='Outra')
plt.legend(loc='center left', fontsize=14)
plt.plot([limiar, limiar], [0, 1], 'r:', linewidth=2)
plt.text(limiar + 0.7, 0.5, 'limiar de decisão', fontsize=14, color='r', ha='center')
plt.show()

In [ ]:
val  = np.array([[1.7], [1.5]])
prob = log_reg.predict_proba(val)
pred = log_reg.predict(val)

print('limiar: ', limiar)
print('valor:', val[0, 0], 'probabilidades:', prob[0], 'previsão:', pred[0])
print('valor:', val[1, 0], 'probabilidades:', prob[1], 'previsão:', pred[1])

O trecho de código e respectiva figura abaixo mostram o mesmo conjunto de dados, mas desta vez exibindo duas características: largura e comprimento da pétala.

Uma vez treinado, o classificador da regressão logística pode estimar a probabilidade de que uma nova flor seja uma _Iris virginica_ com base nessas duas características.

A linha tracejada representa os pontos onde o modelo estima uma probabilidade de 50%: este é o limiar de decisão. Observe que é um limite linear. Cada linha paralela representa os pontos onde o modelo produz uma probabilidade específica.

In [ ]:
from sklearn.linear_model import LogisticRegression

X = iris["data"][:, (2, 3)]  # petal length, petal width
y = (iris["target"] == 2).astype(np.int)

log_reg = LogisticRegression(C=10**10, random_state=42)
log_reg.fit(X, y)

x0, x1 = np.meshgrid(
    np.linspace(2.9, 7, 500).reshape(-1, 1),
    np.linspace(0.8, 2.7, 200).reshape(-1, 1))
X_new = np.c_[x0.ravel(), x1.ravel()]

y_proba = log_reg.predict_proba(X_new)

plt.figure(figsize=(10, 4))
plt.plot(X[y==0, 0], X[y==0, 1], "bs")
plt.plot(X[y==1, 0], X[y==1, 1], "g^")

zz = y_proba[:, 1].reshape(x0.shape)
contour = plt.contour(x0, x1, zz, cmap=plt.cm.brg)

left_right = np.array([2.9, 7])
boundary = -(log_reg.coef_[0][0] * left_right + log_reg.intercept_[0]) / log_reg.coef_[0][1]

plt.clabel(contour, inline=1, fontsize=12)
plt.plot(left_right, boundary, "k--", linewidth=3)
plt.text(6.5, 2.3, "Iris virginica", fontsize=14, color="g", ha="center")
plt.text(3.5, 1.5, "Outra", fontsize=14, color="b", ha="center")
plt.xlabel("Comprimento da pétala", fontsize=14)
plt.ylabel("Largura da pétala", fontsize=14)
plt.axis([2.9, 7, 0.8, 2.7])
plt.show()

# Regressão softmax

A ideia é bastante simples: quando é dada uma instância $\mathbf{x}$, o modelo de regressão softmax primeiro calcula uma pontuação $s_k(\mathbf{x})$ para cada classe $k$. Em seguida, estima a probabilidade de cada classe aplicando a função **softmax** (também chamada de exponencial normalizada) sobre as pontuações.

A classificação será feita para as três espécies de plantas. A classe `LogisticRegression` passa a funcionar como um classificador softmax ao se especificar o parâmetro `multi_class='multinomial'`.

In [ ]:
X = iris['data'][:, (2, 3)]  # comprimento da pétala, largura da pétala
y = iris['target']           # rótulos

softmax_reg = LogisticRegression(multi_class='multinomial', C=10, random_state=42)
softmax_reg.fit(X, y)

A figura abaixo ilustra as regiões de classificação para algumas instâncias, codificadas em três cores.

Além disso, os limiares de probabilidade para a espécie _Iris versicolor_ são mostrados nas curvas contínuas.

In [ ]:
x0, x1 = np.meshgrid(
    np.linspace(0, 8, 500).reshape(-1, 1),
    np.linspace(0, 3.5, 200).reshape(-1, 1),)
X_new = np.c_[x0.ravel(), x1.ravel()]

y_proba = softmax_reg.predict_proba(X_new)
y_predict = softmax_reg.predict(X_new)

zz1 = y_proba[:, 1].reshape(x0.shape)
zz = y_predict.reshape(x0.shape)

plt.figure(figsize=(10, 4))
plt.plot(X[y==2, 0], X[y==2, 1], "g^", label="Iris virginica")
plt.plot(X[y==1, 0], X[y==1, 1], "bs", label="Iris versicolor")
plt.plot(X[y==0, 0], X[y==0, 1], "yo", label="Iris setosa")

from matplotlib.colors import ListedColormap

custom_cmap = ListedColormap(['#fafab0','#9898ff','#a0faa0'])
plt.contourf(x0, x1, zz, cmap=custom_cmap)
contour = plt.contour(x0, x1, zz1, cmap=plt.cm.brg)
plt.clabel(contour, inline=1, fontsize=12)
plt.xlabel("Comprimento da pétala", fontsize=14)
plt.ylabel("Largura da pétala", fontsize=14)
plt.legend(loc="center left", fontsize=14)
plt.axis([0, 7, 0, 3.5])
plt.show()

In [ ]:
val  = np.array([[4, 1]])
prob = softmax_reg.predict_proba(val)
pred = softmax_reg.predict(val)

print('valor:         ', val[0])
print('probabilidades:', prob)
print('previsão:      ', pred[0])